<a href="https://colab.research.google.com/github/Karthikindhu/Machine-Learning-class/blob/main/titani_knn_classifier(nethu).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Steps:**
1. Getting familier with the data
2. Cleaning the data
3. Finding optimal K using cross validation (simple cross validation and KFold cross validation)
4. Finding the accuracy of the knn classifier when we apply on test data 


# **1. Getting familier with the data**

In [ ]:
# Importing related Python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import csv
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Importing the training dataset
df = pd.read_csv('/content/titanic.csv')

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df.shape

(891, 12)

In [ ]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

Dropping the unwanted columns. Name, Ticket, Fare, Cabin seems unwanted columns, so let's remove them.

In [ ]:
df = df.drop('Name', axis=1,)
df = df.drop('Ticket', axis=1,)
#df = df.drop('Fare', axis=1,)
df = df.drop('Cabin', axis=1,)

Combining # siblings and # parents

In [ ]:
df['Family'] = df['SibSp'] + df['Parch'] + 1

In [ ]:
df = df.drop('SibSp', axis=1,)
df = df.drop('Parch', axis=1,)

In [ ]:
df.describe()

,PassengerId,Survived,Pclass,Age,Fare,Family
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,32.204208,1.904602
std,257.353842,0.486592,0.836071,14.526497,49.693429,1.613459
min,1.000000,0.000000,1.000000,0.420000,0.000000,1.000000
25%,223.500000,0.000000,2.000000,20.125000,7.910400,1.000000
50%,446.000000,0.000000,3.000000,28.000000,14.454200,1.000000
75%,668.500000,1.000000,3.000000,38.000000,31.000000,2.000000
max,891.000000,1.000000,3.000000,80.000000,512.329200,11.000000


# 2. Cleaning the data

In [ ]:
df["Age"].isnull().sum()

0

In [ ]:
# By describing data, we found out there few NAN's in Age
# so replacing Age with median of the column
df["Age"] = df["Age"].fillna(df["Age"].mean())

In [ ]:
df.describe()

,PassengerId,Survived,Pclass,Age,Fare,Family
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,32.204208,1.904602
std,257.353842,0.486592,0.836071,13.002015,49.693429,1.613459
min,1.000000,0.000000,1.000000,0.420000,0.000000,1.000000
25%,223.500000,0.000000,2.000000,22.000000,7.910400,1.000000
50%,446.000000,0.000000,3.000000,29.699118,14.454200,1.000000
75%,668.500000,1.000000,3.000000,35.000000,31.000000,2.000000
max,891.000000,1.000000,3.000000,80.000000,512.329200,11.000000


In [ ]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [ ]:
#finding NAN's in Embarked column
df['Embarked'].isna().sum()

2

In [ ]:
#Replacing the NAN's with most frequently used one i.e mode(metric that gives us most frequently used value)
print(df["Embarked"].mode())
df["Embarked"] = df["Embarked"].fillna("S")

0    S
dtype: object


In [ ]:
df['Embarked'].describe()

count     891
unique      3
top         S
freq      646
Name: Embarked, dtype: object

In [ ]:
# Replacing the categorical value Embarked into numerical value
df["Embarked"].unique()

array(['S', 'C', 'Q'], dtype=object)

In [ ]:
df.Embarked.replace(['S', 'C', 'Q'], [1, 2, 3], inplace=True)

In [ ]:
df.Sex.replace(['male', 'female'], [1,0], inplace=True)

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Family
0,1,0,3,1,22.0,7.2500,1,2
1,2,1,1,0,38.0,71.2833,2,2
2,3,1,3,0,26.0,7.9250,1,1
3,4,1,1,0,35.0,53.1000,1,2
4,5,0,3,1,35.0,8.0500,1,1


# 3. Finding optimal K using cross validation (simple cross validation and KFold cross validation)

In [ ]:
# importing libraries
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from collections import Counter
from sklearn.model_selection import cross_validate

In [ ]:
X = np.array(df.filter(['Pclass','Sex','Embarked','Family','Age'], axis=1))

In [ ]:
y = np.array(df.filter(['Survived'], axis=1))

* Splitting the data into train and test data.          
* Splitting the train data into train and validation data.

In [ ]:
# simple cross validation
X_1, X_test, y_1, y_test = train_test_split(X,y, test_size=0.3,random_state=7)
X_tr, X_cv, y_tr, y_cv = train_test_split(X_1, y_1, test_size=0.3,random_state=7)

Getting the best k using simple cross validation.

In [ ]:
final_scores = []
for i in range(1,30,2):
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(X_tr, y_tr)
    pred = knn.predict(X_cv)
    acc = accuracy_score(y_cv, pred, normalize=True) * float(100)
    final_scores.append(acc)
    print('\n CV accuracy for k=%d is %d'%(i,acc))


 CV accuracy for k=1 is 80

 CV accuracy for k=3 is 75

 CV accuracy for k=5 is 74

 CV accuracy for k=7 is 73

 CV accuracy for k=9 is 73

 CV accuracy for k=11 is 71

 CV accuracy for k=13 is 71

 CV accuracy for k=15 is 70

 CV accuracy for k=17 is 70

 CV accuracy for k=19 is 68

 CV accuracy for k=21 is 67

 CV accuracy for k=23 is 67

 CV accuracy for k=25 is 66

 CV accuracy for k=27 is 64

 CV accuracy for k=29 is 63


In [ ]:
optimal_k = final_scores.index(max(final_scores))
print(optimal_k)

1
